In [1]:
!unzip archive.zip

Archive:  archive.zip
  inflating: abcnews-date-text.csv   


In [49]:
import pandas as pd

In [50]:
df = pd.read_csv('abcnews-date-text.csv')

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226258 entries, 0 to 1226257
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   publish_date   1226258 non-null  int64 
 1   headline_text  1226258 non-null  object
dtypes: int64(1), object(1)
memory usage: 18.7+ MB


In [52]:
df.head(3)

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit


In [54]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem import PorterStemmer 
import numpy as np
np.random.seed(2018)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zhivarsourati/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [55]:
def lemmatize_stemming(text):
    ps = PorterStemmer() 
    return ps.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [28]:
doc_sample = df.iloc[4310].values[1]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [29]:
processed_docs = df['headline_text'].map(preprocess)

In [30]:
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [32]:
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [33]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [34]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [35]:
bow_corpus[400]

[(309, 1), (1034, 1), (1035, 1), (1036, 1), (1037, 1)]

In [40]:
bow_doc_400 = bow_corpus[400]

for i in range(len(bow_doc_400)):
    print("Word {} (\"{}\") appears {} time.".format(
        bow_doc_400[i][0], 
        dictionary[bow_doc_400[i][0]], 
        bow_doc_400[i][1])
     )

Word 309 ("scare") appears 1 time.
Word 1034 ("fume") appears 1 time.
Word 1035 ("paint") appears 1 time.
Word 1036 ("reopen") appears 1 time.
Word 1037 ("school") appears 1 time.


In [45]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5852942020878993),
 (1, 0.38405854933668493),
 (2, 0.5017732999224691),
 (3, 0.5080878695349914)]


In [46]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [47]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"donald" + 0.029*"kill" + 0.027*"market" + 0.021*"border" + 0.018*"attack" + 0.016*"scott" + 0.014*"bank" + 0.013*"amid" + 0.013*"australia" + 0.011*"share"
Topic: 1 
Words: 0.040*"polic" + 0.040*"sydney" + 0.020*"die" + 0.019*"death" + 0.017*"shoot" + 0.016*"hous" + 0.016*"rise" + 0.016*"investig" + 0.013*"melbourn" + 0.012*"break"
Topic: 2 
Words: 0.017*"speak" + 0.017*"countri" + 0.015*"releas" + 0.014*"work" + 0.012*"video" + 0.012*"campaign" + 0.012*"media" + 0.011*"launch" + 0.011*"bodi" + 0.010*"protest"
Topic: 3 
Words: 0.044*"trump" + 0.023*"world" + 0.020*"australia" + 0.019*"open" + 0.017*"women" + 0.015*"australian" + 0.015*"final" + 0.014*"miss" + 0.012*"win" + 0.011*"take"
Topic: 4 
Words: 0.022*"bushfir" + 0.021*"school" + 0.019*"nation" + 0.017*"chang" + 0.015*"leav" + 0.012*"street" + 0.011*"rural" + 0.011*"darwin" + 0.010*"climat" + 0.010*"show"
Topic: 5 
Words: 0.038*"elect" + 0.027*"live" + 0.017*"state" + 0.016*"victorian" + 0.016*"time" + 0.

In [48]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

Topic: 0 Word: 0.028*"trump" + 0.011*"coronaviru" + 0.011*"royal" + 0.009*"commiss" + 0.009*"vaccin" + 0.009*"friday" + 0.009*"monday" + 0.008*"michael" + 0.007*"lockdown" + 0.007*"covid"
Topic: 1 Word: 0.020*"crash" + 0.010*"interview" + 0.009*"fatal" + 0.008*"climat" + 0.007*"plead" + 0.007*"truck" + 0.007*"peter" + 0.007*"die" + 0.007*"light" + 0.006*"septemb"
Topic: 2 Word: 0.017*"news" + 0.013*"market" + 0.009*"gold" + 0.009*"australian" + 0.008*"coast" + 0.008*"rural" + 0.007*"world" + 0.007*"final" + 0.006*"financ" + 0.006*"wall"
Topic: 3 Word: 0.010*"christma" + 0.009*"coronaviru" + 0.009*"weather" + 0.007*"pandem" + 0.006*"say" + 0.006*"govern" + 0.006*"elect" + 0.005*"australia" + 0.005*"brexit" + 0.005*"china"
Topic: 4 Word: 0.017*"coronaviru" + 0.017*"donald" + 0.016*"covid" + 0.015*"countri" + 0.010*"hour" + 0.009*"restrict" + 0.008*"north" + 0.008*"south" + 0.008*"wednesday" + 0.007*"updat"
Topic: 5 Word: 0.017*"polic" + 0.017*"charg" + 0.016*"murder" + 0.012*"court" + 0.

In [56]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWord: {}'.format(idx, topic))

Topic: 0 
Word: 0.028*"trump" + 0.011*"coronaviru" + 0.011*"royal" + 0.009*"commiss" + 0.009*"vaccin" + 0.009*"friday" + 0.009*"monday" + 0.008*"michael" + 0.007*"lockdown" + 0.007*"covid"
Topic: 1 
Word: 0.020*"crash" + 0.010*"interview" + 0.009*"fatal" + 0.008*"climat" + 0.007*"plead" + 0.007*"truck" + 0.007*"peter" + 0.007*"die" + 0.007*"light" + 0.006*"septemb"
Topic: 2 
Word: 0.017*"news" + 0.013*"market" + 0.009*"gold" + 0.009*"australian" + 0.008*"coast" + 0.008*"rural" + 0.007*"world" + 0.007*"final" + 0.006*"financ" + 0.006*"wall"
Topic: 3 
Word: 0.010*"christma" + 0.009*"coronaviru" + 0.009*"weather" + 0.007*"pandem" + 0.006*"say" + 0.006*"govern" + 0.006*"elect" + 0.005*"australia" + 0.005*"brexit" + 0.005*"china"
Topic: 4 
Word: 0.017*"coronaviru" + 0.017*"donald" + 0.016*"covid" + 0.015*"countri" + 0.010*"hour" + 0.009*"restrict" + 0.008*"north" + 0.008*"south" + 0.008*"wednesday" + 0.007*"updat"
Topic: 5 
Word: 0.017*"polic" + 0.017*"charg" + 0.016*"murder" + 0.012*"court